In [1]:
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import os
import numpy as np
import pandas as pd

# テキストファイルをnumpy配列に読み込み
all_data = np.loadtxt("claim_toiawase_data_python.txt")
#numpy配列をDataFrame型に変換(でーたを見やすくするため)
all_data_pd = pd.DataFrame(all_data)
#DataFrame型の列名を設定
all_data_pd.columns = ["seq", "Imp", "Imp_val", "m_sum_neg", "c_neg", "m_sum_all", "m_sum_nzr", "Mag", "c_nzr", "c_all", "m_max", "s_min", "m_sdv_neg", 
                   "s_sdv_neg", "m_avg_neg", "s_avg_neg", "s_sdv_nzr", "m_sdv_all", "s_sdv_all", "s_avg_nzr", "m_avg_nzr", "s_avg_all", "Sc",
                   "m_avg_all", "m_sum_pos", "c_pos", "m_min", "s_max", "m_sdv_pos", "s_sdv_pos", "m_avg_pos", "s_avg_pos", "m_sdv_nzr"]
print(all_data_pd["seq"])   #ImpとImp_valは同じ

# 特徴量の部分のみall_Xに読み込み
all_X = all_data[:9813,3:33]

#ラベル部分をall_yに読み込み
all_y = all_data[:9813, 2]

#文書番号をall_numに読み込み
all_num = np.array(all_data_pd["seq"], dtype= "int64")

print(all_num)



0          1.0
1          2.0
2          3.0
3          4.0
4          5.0
5          6.0
6          7.0
7          8.0
8          9.0
9         10.0
10        11.0
11        12.0
12        13.0
13        14.0
14        15.0
15        16.0
16        17.0
17        18.0
18        19.0
19        20.0
20        21.0
21        22.0
22        23.0
23        24.0
24        25.0
25        26.0
26        27.0
27        28.0
28        29.0
29        30.0
         ...  
9783    9784.0
9784    9785.0
9785    9786.0
9786    9787.0
9787    9788.0
9788    9789.0
9789    9790.0
9790    9791.0
9791    9792.0
9792    9793.0
9793    9794.0
9794    9795.0
9795    9796.0
9796    9797.0
9797    9798.0
9798    9799.0
9799    9800.0
9800    9801.0
9801    9802.0
9802    9803.0
9803    9804.0
9804    9805.0
9805    9806.0
9806    9807.0
9807    9808.0
9808    9809.0
9809    9810.0
9810    9811.0
9811    9812.0
9812    9813.0
Name: seq, Length: 9813, dtype: float64
[   1    2    3 ... 9811 9812 9813]


In [2]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
for f in [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ,13, 14, 15, 16, 17, 18]:
    all_X = all_data[:9813,3:33]
    all_X = all_X[::, :f]

    for c in [0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0]:
        for g in [1.0, 0.1, 0.01, 0.001, 0.0001, 0.00001]:
            i = 1
            #十回分のそれぞれのスコアを格納する空のlistを作成
            accuracy_train = []
            f1_train = []
            precision_train = []
            recall_train = []

            accuracy_test = []
            f1_test = []
            precision_test = []
            recall_test = []
            
            #パイプラインの作成
            pipe = make_pipeline(StandardScaler(), SVC(C=c, gamma=g, kernel="rbf", random_state=1))
            #層化10-fold交差検証の開始
            kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=2)
            for train_index, test_index in kf.split(all_X, all_y):
                #繰り返し毎に訓練データとテストデータと文書番号の分割
                train_X, test_X = all_X[train_index], all_X[test_index]
                train_y, test_y = all_y[train_index], all_y[test_index]
                train_num, test_num = all_num[train_index], all_num[test_index]
                # 訓練データのラベル1のsmote前のデータ数の出力
                #print("train_X(label=1): %s" %train_X[train_y == 1].shape[0])
                #print("train_X(label=0): %s" %train_X[train_y == 0].shape[0])
                smote = SMOTE("minority", random_state=2)
                #train_X_smote , train_y_smote = smote.fit_sample(train_X, train_y)
                #print("smote_train_X(label=1): %s" %train_X_smote[ train_y_smote ==1 ].shape[0])
                #print("smote_train_X(label=0): %s" %train_X_smote[ train_y_smote ==0 ].shape[0])
                enn = EditedNearestNeighbours("all",random_state=2)
                #train_X_enn , train_y_enn = enn.fit_sample(train_X, train_y)
                #print("enn_train_X(label=1): %s" %train_X_enn[ train_y_enn ==1 ].shape[0])
                #print("enn_train_X(label=0): %s" %train_X_enn[ train_y_enn ==0 ].shape[0])
                #SMOTEENNを行い新しい訓練データを作成

                train_X_smoteenn , train_y_smoteenn = SMOTEENN(random_state=2, smote=smote,enn=enn ).fit_sample(train_X, train_y)
                #print("smoteenn_train_X(label=1): %s" %train_X_smoteenn[ train_y_smoteenn ==1 ].shape[0])
                #print("smoteenn_train_X(label=0): %s" %train_X_smoteenn[ train_y_smoteenn ==0 ].shape[0])
                
                #パイプを訓練データに適合
                pipe.fit(train_X_smoteenn, train_y_smoteenn)
                #訓練データを入力した時の予測ラベルをpre_label_trainに読み込む
                pre_label_train = pipe.predict(train_X_smoteenn)
                
                #訓練データにおける評価の実施
                accuracy_train.append(accuracy_score(train_y_smoteenn, pre_label_train))
                f1_train.append(f1_score(train_y_smoteenn, pre_label_train))
                precision_train.append(precision_score(train_y_smoteenn, pre_label_train))
                recall_train.append(recall_score(train_y_smoteenn, pre_label_train))

                #テストデータを入力した時の予測ラベルをpre_label_testに読み込む
                pre_label_test = pipe.predict(test_X)
       
    
                #テストデータにおける評価の実施
                accuracy_test.append(accuracy_score(test_y, pre_label_test))
                f1_test.append(f1_score(test_y, pre_label_test))
                precision_test.append(precision_score(test_y, pre_label_test))
                recall_test.append(recall_score(test_y, pre_label_test))
            
                #予測ラベルに対応する文書番号の結合
                V_test_num = test_num.reshape(len(test_num), 1)
                V_pre_label_test = pre_label_test.reshape(len(pre_label_test), 1)
                num_and_y = np.hstack((V_test_num, V_pre_label_test))

                #交差検証の何個目を行っているのかの出力
                #print(i)
                
                #csvファイルに出力
                np.savetxt("%d_SMOTE_num_and_prelabel.csv" %i, num_and_y, fmt="%3.f", delimiter="\t", header="文書番号\t予測ラベル")
                i = i + 1

            #十回のスコアの平均と標準偏差の出力
            print("\n")
            print("変数の数:%s      C:%s      gamma:%s" %(f,c,g))
            print("accuracy_train: %.4f +- %.4f" %(np.mean(accuracy_train), np.std(accuracy_train)))
            print("f1_train: %.4f +- %.4f" %(np.mean(f1_train), np.std(f1_train)))
            print("precision_train: %.4f +- %.4f" %(np.mean(precision_train), np.std(precision_train)))
            print("recall_train: %.4f +- %.4f" %(np.mean(recall_train), np.std(recall_train)))

            print("accuracy_test: %.4f +- %.4f" %(np.mean(accuracy_test), np.std(accuracy_test)))
            print("f1_test: %.4f +- %.4f" %(np.mean(f1_test), np.std(f1_test)))
            print("precision_test: %.4f +- %.4f" %(np.mean(precision_test), np.std(precision_test)))
            print("recall_test: %.4f +- %.4f" %(np.mean(recall_test), np.std(recall_test)))
            print("list_shape: %s" %len(accuracy_train))
            print("\n")
            
            



変数の数:2      C:0.1      gamma:1.0
accuracy_train: 0.9810 +- 0.0204
f1_train: 0.9757 +- 0.0256
precision_train: 0.9751 +- 0.0176
recall_train: 0.9770 +- 0.0396
accuracy_test: 0.8061 +- 0.0154
f1_test: 0.4788 +- 0.0105
precision_test: 0.3938 +- 0.0244
recall_test: 0.6172 +- 0.0471
list_shape: 10




変数の数:2      C:0.1      gamma:0.1
accuracy_train: 0.9737 +- 0.0230
f1_train: 0.9655 +- 0.0305
precision_train: 0.9762 +- 0.0130
recall_train: 0.9561 +- 0.0509
accuracy_test: 0.8257 +- 0.0156
f1_test: 0.4943 +- 0.0198
precision_test: 0.4288 +- 0.0331
recall_test: 0.5897 +- 0.0476
list_shape: 10




変数の数:2      C:0.1      gamma:0.01
accuracy_train: 0.9441 +- 0.0243
f1_train: 0.9232 +- 0.0334
precision_train: 0.9836 +- 0.0103
recall_train: 0.8712 +- 0.0543
accuracy_test: 0.8564 +- 0.0081
f1_test: 0.4895 +- 0.0243
precision_test: 0.5045 +- 0.0289
recall_test: 0.4780 +- 0.0402
list_shape: 10




変数の数:2      C:0.1      gamma:0.001
accuracy_train: 0.8206 +- 0.0201
f1_train: 0.6972 +- 0.0456
precisio

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio



変数の数:2      C:0.1      gamma:0.0001
accuracy_train: 0.6147 +- 0.0179
f1_train: 0.0114 +- 0.0047
precision_train: 1.0000 +- 0.0000
recall_train: 0.0057 +- 0.0024
accuracy_test: 0.8559 +- 0.0006
f1_test: 0.0028 +- 0.0056
precision_test: 0.2000 +- 0.4000
recall_test: 0.0014 +- 0.0028
list_shape: 10




/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio

...



変数の数:17      C:0.1      gamma:1e-05
accuracy_train: 0.5301 +- 0.0129
f1_train: 0.0006 +- 0.0008
precision_train: 0.6000 +- 0.4899
recall_train: 0.0003 +- 0.0004
accuracy_test: 0.8558 +- 0.0006
f1_test: 0.0014 +- 0.0042
precision_test: 0.1000 +- 0.3000
recall_test: 0.0007 +- 0.0021
list_shape: 10




変数の数:17      C:1.0      gamma:1.0
accuracy_train: 0.9511 +- 0.0039
f1_train: 0.9479 +- 0.0034
precision_train: 0.9503 +- 0.0066
recall_train: 0.9455 +- 0.0049
accuracy_test: 0.7816 +- 0.0336
f1_test: 0.4695 +- 0.0317
precision_test: 0.3667 +- 0.0421
recall_test: 0.6631 +- 0.0454
list_shape: 10




変数の数:17      C:1.0      gamma:0.1
accuracy_train: 0.9209 +- 0.0044
f1_train: 0.9149 +- 0.0042
precision_train: 0.9256 +- 0.0056
recall_train: 0.9045 +- 0.0074
accuracy_test: 0.8158 +- 0.0086
f1_test: 0.5012 +- 0.0228
precision_test: 0.4116 +- 0.0172
recall_test: 0.6426 +- 0.0468
list_shape: 10




変数の数:17      C:1.0      gamma:0.01
accuracy_train: 0.9110 +- 0.0048
f1_train: 0.9049 +- 0.0046
prec



変数の数:17      C:10000.0      gamma:0.001
accuracy_train: 0.9185 +- 0.0039
f1_train: 0.9124 +- 0.0035
precision_train: 0.9226 +- 0.0069
recall_train: 0.9026 +- 0.0092
accuracy_test: 0.8140 +- 0.0116
f1_test: 0.5018 +- 0.0204
precision_test: 0.4100 +- 0.0182
recall_test: 0.6497 +- 0.0485
list_shape: 10




変数の数:17      C:10000.0      gamma:0.0001
accuracy_train: 0.9093 +- 0.0058
f1_train: 0.9033 +- 0.0045
precision_train: 0.9059 +- 0.0072
recall_train: 0.9008 +- 0.0081
accuracy_test: 0.8045 +- 0.0091
f1_test: 0.4888 +- 0.0179
precision_test: 0.3931 +- 0.0151
recall_test: 0.6483 +- 0.0422
list_shape: 10




変数の数:17      C:10000.0      gamma:1e-05
accuracy_train: 0.9084 +- 0.0055
f1_train: 0.9027 +- 0.0042
precision_train: 0.9018 +- 0.0078
recall_train: 0.9037 +- 0.0093
accuracy_test: 0.8017 +- 0.0108
f1_test: 0.4867 +- 0.0196
precision_test: 0.3891 +- 0.0179
recall_test: 0.6518 +- 0.0420
list_shape: 10




変数の数:18      C:0.1      gamma:1.0
accuracy_train: 0.9234 +- 0.0053
f1_train: 0.919

/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/iwanaga/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precisio



変数の数:18      C:0.1      gamma:1e-05
accuracy_train: 0.5321 +- 0.0137
f1_train: 0.0683 +- 0.2019
precision_train: 0.6508 +- 0.4497
recall_train: 0.1004 +- 0.2999
accuracy_test: 0.7845 +- 0.2135
f1_test: 0.0252 +- 0.0755
precision_test: 0.0144 +- 0.0432
recall_test: 0.1000 +- 0.3000
list_shape: 10




変数の数:18      C:1.0      gamma:1.0
accuracy_train: 0.9539 +- 0.0039
f1_train: 0.9509 +- 0.0034
precision_train: 0.9523 +- 0.0047
recall_train: 0.9496 +- 0.0054
accuracy_test: 0.7732 +- 0.0298
f1_test: 0.4606 +- 0.0316
precision_test: 0.3545 +- 0.0391
recall_test: 0.6659 +- 0.0439
list_shape: 10




変数の数:18      C:1.0      gamma:0.1
accuracy_train: 0.9211 +- 0.0052
f1_train: 0.9152 +- 0.0044
precision_train: 0.9244 +- 0.0042
recall_train: 0.9061 +- 0.0070
accuracy_test: 0.8118 +- 0.0102
f1_test: 0.4973 +- 0.0215
precision_test: 0.4052 +- 0.0175
recall_test: 0.6461 +- 0.0483
list_shape: 10




変数の数:18      C:1.0      gamma:0.01
accuracy_train: 0.9127 +- 0.0065
f1_train: 0.9065 +- 0.0058
prec



変数の数:18      C:10000.0      gamma:0.001
accuracy_train: 0.9182 +- 0.0045
f1_train: 0.9121 +- 0.0037
precision_train: 0.9208 +- 0.0055
recall_train: 0.9036 +- 0.0097
accuracy_test: 0.8129 +- 0.0086
f1_test: 0.4989 +- 0.0203
precision_test: 0.4069 +- 0.0158
recall_test: 0.6462 +- 0.0426
list_shape: 10




変数の数:18      C:10000.0      gamma:0.0001
accuracy_train: 0.9093 +- 0.0063
f1_train: 0.9033 +- 0.0049
precision_train: 0.9054 +- 0.0053
recall_train: 0.9012 +- 0.0095
accuracy_test: 0.8037 +- 0.0089
f1_test: 0.4903 +- 0.0185
precision_test: 0.3924 +- 0.0154
recall_test: 0.6546 +- 0.0364
list_shape: 10




変数の数:18      C:10000.0      gamma:1e-05
accuracy_train: 0.9075 +- 0.0057
f1_train: 0.9017 +- 0.0040
precision_train: 0.9009 +- 0.0088
recall_train: 0.9027 +- 0.0103
accuracy_test: 0.8013 +- 0.0104
f1_test: 0.4866 +- 0.0191
precision_test: 0.3884 +- 0.0176
recall_test: 0.6525 +- 0.0315
list_shape: 10


